# Starbucks Capstone Challenge

### Introduction

This data set contains simulated data that mimics customer behavior on the Starbucks rewards mobile app. Once every few days, Starbucks sends out an offer to users of the mobile app. An offer can be merely an advertisement for a drink or an actual offer such as a discount or BOGO (buy one get one free). Some users might not receive any offer during certain weeks. 

Not all users receive the same offer, and that is the challenge to solve with this data set.

Your task is to combine transaction, demographic and offer data to determine which demographic groups respond best to which offer type. This data set is a simplified version of the real Starbucks app because the underlying simulator only has one product whereas Starbucks actually sells dozens of products.

Every offer has a validity period before the offer expires. As an example, a BOGO offer might be valid for only 5 days. You'll see in the data set that informational offers have a validity period even though these ads are merely providing information about a product; for example, if an informational offer has 7 days of validity, you can assume the customer is feeling the influence of the offer for 7 days after receiving the advertisement.

You'll be given transactional data showing user purchases made on the app including the timestamp of purchase and the amount of money spent on a purchase. This transactional data also has a record for each offer that a user receives as well as a record for when a user actually views the offer. There are also records for when a user completes an offer. 

Keep in mind as well that someone using the app might make a purchase through the app without having received an offer or seen an offer.

### Example

To give an example, a user could receive a discount offer buy 10 dollars get 2 off on Monday. The offer is valid for 10 days from receipt. If the customer accumulates at least 10 dollars in purchases during the validity period, the customer completes the offer.

However, there are a few things to watch out for in this data set. Customers do not opt into the offers that they receive; in other words, a user can receive an offer, never actually view the offer, and still complete the offer. For example, a user might receive the "buy 10 dollars get 2 dollars off offer", but the user never opens the offer during the 10 day validity period. The customer spends 15 dollars during those ten days. There will be an offer completion record in the data set; however, the customer was not influenced by the offer because the customer never viewed the offer.

### Cleaning

This makes data cleaning especially important and tricky.

You'll also want to take into account that some demographic groups will make purchases even if they don't receive an offer. From a business perspective, if a customer is going to make a 10 dollar purchase without an offer anyway, you wouldn't want to send a buy 10 dollars get 2 dollars off offer. You'll want to try to assess what a certain demographic group will buy when not receiving any offers.

### Final Advice

Because this is a capstone project, you are free to analyze the data any way you see fit. For example, you could build a machine learning model that predicts how much someone will spend based on demographics and offer type. Or you could build a model that predicts whether or not someone will respond to an offer. Or, you don't need to build a machine learning model at all. You could develop a set of heuristics that determine what offer you should send to each customer (i.e., 75 percent of women customers who were 35 years old responded to offer A vs 40 percent from the same demographic to offer B, so send offer A).

# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

**Note:** If you are using the workspace, you will need to go to the terminal and run the command `conda update pandas` before reading in the files. This is because the version of pandas in the workspace cannot read in the transcript.json file correctly, but the newest version of pandas can. You can access the termnal from the orange icon in the top left of this notebook.  

You can see how to access the terminal and how the install works using the two images below.  First you need to access the terminal:

<img src="pic1.png"/>

Then you will want to run the above command:

<img src="pic2.png"/>

Finally, when you enter back into the notebook (use the jupyter icon again), you should be able to run the below cell without any errors.

In [1]:
import pandas as pd
import numpy as np
import math
import json
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from matplotlib.ticker import MaxNLocator

# read in the json files
portfolio_raw = pd.read_json('data/portfolio.json', orient='records', lines=True)
profile_raw = pd.read_json('data/profile.json', orient='records', lines=True)
transcript_raw = pd.read_json('data/transcript.json', orient='records', lines=True)

## Data understanding



**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)


** -> Data is cleaned and ready to be used**

In [ ]:
portfolio_raw.info()

In [ ]:
# portfolio_raw['channels'].explode().unique()

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**-> the NoneType gender will be removed from profile dataset and storage in a new dataset**

In [ ]:
profile_raw.info()

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record

In [ ]:
transcript_raw.info()

In [ ]:
transcript_raw['event'].value_counts()

In [ ]:
{[*x][0] for x in transcript_raw['value']}

### Data Cleaning and Formatting Plan

#### **Objective**
- Clean and standardize the dataset while handling null values appropriately.

---

#### **Profile Dataset**
1. Split the `profile` dataset into two subsets:
   - One containing rows where the `gender` is `none`.
   - Another containing rows with all other `gender` values.

2. Convert the `became_member_on` column to a standardized **datetime** format for consistency and easier analysis.

---

#### **Portfolio Dataset**
- Update column names to make them more descriptive and easier to read.

---

#### **Transcript Dataset**
- Modify the keys within the `value` dictionaries by replacing spaces (`' '`) with underscores (`'_'`) to ensure uniformity and avoid potential issues in further processing.

---

In [2]:
profile = profile_raw.copy(deep=True)

# Convert the 'became_member_on' column to a datetime format
profile['became_member_on'] = pd.to_datetime(profile['became_member_on'], format='%Y%m%d')

# Create a new column with only the year and month of the membership
profile['bec_memb_year_month'] = pd.to_datetime(profile['became_member_on'], format='%Y%m%d').dt.strftime('%Y-%m')

In [ ]:
profile.info()

In [3]:
portfolio = portfolio_raw.copy()
portfolio.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reward      10 non-null     int64 
 1   channels    10 non-null     object
 2   difficulty  10 non-null     int64 
 3   duration    10 non-null     int64 
 4   offer_type  10 non-null     object
 5   id          10 non-null     object
dtypes: int64(3), object(3)
memory usage: 612.0+ bytes


In [4]:
# Standardisation of column names
transcript = transcript_raw.copy(deep=True)
transcript['event'] = transcript['event'].str.replace(' ','_')

# Fixing the dicts key names
def fix_offer_id(value):
    if isinstance(value, dict) and 'offer id' in value:
        value['offer_id'] = value.pop('offer id')
    return value

transcript['value'] = transcript['value'].apply(fix_offer_id)

# Normalizing the value column and create a new dataframe
value_df = pd.json_normalize(transcript['value']) 
transcript_b = pd.concat([transcript, value_df], axis=1).drop('value', axis=1)

transcript_b.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   person    306534 non-null  object 
 1   event     306534 non-null  object 
 2   time      306534 non-null  int64  
 3   offer_id  167581 non-null  object 
 4   amount    138953 non-null  float64
 5   reward    33579 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 14.0+ MB


In [5]:
transactions = transcript_b.loc[transcript_b['event'] == 'transaction',:]
transcript_c = transcript_b.loc[transcript_b['event'] != 'transaction',:]
transcript_c.info()

<class 'pandas.core.frame.DataFrame'>
Index: 167581 entries, 0 to 306527
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   person    167581 non-null  object 
 1   event     167581 non-null  object 
 2   time      167581 non-null  int64  
 3   offer_id  167581 non-null  object 
 4   amount    0 non-null       float64
 5   reward    33579 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 8.9+ MB


In [ ]:
transactions.info()

In [6]:
transcript_collection = transcript_c.merge(profile, left_on='person', right_on='id', how='left').drop('id', axis=1)

In [7]:
transcript_collection.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167581 entries, 0 to 167580
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   person               167581 non-null  object        
 1   event                167581 non-null  object        
 2   time                 167581 non-null  int64         
 3   offer_id             167581 non-null  object        
 4   amount               0 non-null       float64       
 5   reward               33579 non-null   float64       
 6   gender               148805 non-null  object        
 7   age                  167581 non-null  int64         
 8   became_member_on     167581 non-null  datetime64[ns]
 9   income               148805 non-null  float64       
 10  bec_memb_year_month  167581 non-null  object        
dtypes: datetime64[ns](1), float64(3), int64(2), object(5)
memory usage: 14.1+ MB


In [8]:
transcript_collection = transcript_collection.merge(portfolio, left_on='offer_id', right_on='id', how='left', suffixes=(' ', '_std')).drop('id', axis=1)

transcript_collection.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167581 entries, 0 to 167580
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   person               167581 non-null  object        
 1   event                167581 non-null  object        
 2   time                 167581 non-null  int64         
 3   offer_id             167581 non-null  object        
 4   amount               0 non-null       float64       
 5   reward               33579 non-null   float64       
 6   gender               148805 non-null  object        
 7   age                  167581 non-null  int64         
 8   became_member_on     167581 non-null  datetime64[ns]
 9   income               148805 non-null  float64       
 10  bec_memb_year_month  167581 non-null  object        
 11  reward_std           167581 non-null  int64         
 12  channels             167581 non-null  object        
 13  difficulty    

In [9]:
port_id = {
    'ae264e3637204a6fb9bb56bc8210ddfd': 'ofr_A',
    '4d5c57ea9a6940dd891ad53e9dbe8da0': 'ofr_B',
    '3f207df678b143eea3cee63160fa8bed': 'ofr_C',
    '9b98b8c7a33c4b65b9aebfe6a799e6d9': 'ofr_D',
    '0b1e1539f2cc45b7b9fa7c272da2e1d7': 'ofr_E',
    '2298d6c36e964ae4a3e7e9706d1fb8c2': 'ofr_F',
    'fafdcd668e3743c1bb461111dcafc2a4': 'ofr_G',
    '5a8bc65990b245e5a138643cd4eb9837': 'ofr_H',
    'f19421c1d4aa40978ebb69ca19b0e20d': 'ofr_I',
    '2906b810c7d4411798c6938adc9daaa5': 'ofr_J'
}

transcript_collection['ofr_id_short'] = transcript_collection['offer_id'].map(port_id)
transcript_collection = transcript_collection.drop(['offer_id'], axis=1)

In [10]:
transcript_collection

,person,event,time,amount,reward,gender,age,became_member_on,income,bec_memb_year_month,reward_std,channels,difficulty,duration,offer_type,ofr_id_short
0,78afa995795e4d85b5d9ceeca43f5fef,offer_received,0,NaN,NaN,F,75,2017-05-09,100000.0,2017-05,5,"[web, email, mobile]",5,7,bogo,ofr_D
1,a03223e636434f42ac4c3df47e8bac43,offer_received,0,NaN,NaN,None,118,2017-08-04,NaN,2017-08,5,"[web, email]",20,10,discount,ofr_E
2,e2127556f4f64592b11af22de27a7932,offer_received,0,NaN,NaN,M,68,2018-04-26,70000.0,2018-04,2,"[web, email, mobile]",10,7,discount,ofr_J
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer_received,0,NaN,NaN,None,118,2017-09-25,NaN,2017-09,2,"[web, email, mobile, social]",10,10,discount,ofr_G
4,68617ca6246f4fbc85e91a2a49552598,offer_received,0,NaN,NaN,None,118,2017-10-02,NaN,2017-10,10,"[web, email, mobile, social]",10,5,bogo,ofr_B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167576,a6f84f4e976f44508c358cc9aba6d2b3,offer_completed,714,NaN,3.0,None,118,2017-01-16,NaN,2017-01,3,"[web, email, mobile, social]",7,7,discount,ofr_F
167577,b895c57e8cd047a8872ce02aa54759d6,offer_completed,714,NaN,2.0,None,118,2017-01-25,NaN,2017-01,2,"[web, email, mobile, social]",10,10,discount,ofr_G
167578,8dda575c2a1d44b9ac8e8b07b93d1f8e,offer_viewed,714,NaN,NaN,F,60,2017-09-08,64000.0,2017-09,5,"[web, email]",20,10,discount,ofr_E
167579,8431c16f8e1d440880db371a68f82dd0,offer_completed,714,NaN,2.0,M,39,2018-06-27,39000.0,2018-06,2,"[web, email, mobile, social]",10,10,discount,ofr_G


In [11]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 138953 entries, 12654 to 306533
Data columns (total 6 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   person    138953 non-null  object 
 1   event     138953 non-null  object 
 2   time      138953 non-null  int64  
 3   offer_id  0 non-null       object 
 4   amount    138953 non-null  float64
 5   reward    0 non-null       float64
dtypes: float64(2), int64(1), object(3)
memory usage: 7.4+ MB


In [12]:
# MISSING

transactions = transactions.dropna(axis=1)

In [13]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 138953 entries, 12654 to 306533
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   person  138953 non-null  object 
 1   event   138953 non-null  object 
 2   time    138953 non-null  int64  
 3   amount  138953 non-null  float64
dtypes: float64(1), int64(1), object(2)
memory usage: 5.3+ MB


In [14]:
transcript_collection.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 167581 entries, 0 to 167580
Data columns (total 16 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   person               167581 non-null  object        
 1   event                167581 non-null  object        
 2   time                 167581 non-null  int64         
 3   amount               0 non-null       float64       
 4   reward               33579 non-null   float64       
 5   gender               148805 non-null  object        
 6   age                  167581 non-null  int64         
 7   became_member_on     167581 non-null  datetime64[ns]
 8   income               148805 non-null  float64       
 9   bec_memb_year_month  167581 non-null  object        
 10  reward_std           167581 non-null  int64         
 11  channels             167581 non-null  object        
 12  difficulty           167581 non-null  int64         
 13  duration      

In [15]:
transcript_collection['gender'].value_counts(dropna=False)

gender
M       82896
F       63719
None    18776
O        2190
Name: count, dtype: int64